In [ ]:
pip install PyMuPDF

In [ ]:
# 의사록 pdf 다운받기
import requests
from bs4 import BeautifulSoup
import os

if not os.path.exists('pdf_files'):
    os.makedirs('pdf_files')

for page_num in range(1, 41): 
    URL = f'https://www.bok.or.kr/portal/bbs/B0000245/list.do?menuNo=200761&pageIndex={page_num}'
    resp = requests.get(URL)
    soup = BeautifulSoup(resp.text, 'html.parser')
    
    li_list = []
    li = soup.find('div', class_='bdLine type2').find('ul').find('li')
    li_li = li.find_next_siblings('li')
    li_list.append(li)
    li_list.extend(li_li)
    
    for j in range(0, len(li_list)):
        try:
            link = li_list[j].find('div', class_='fileGoupBox').find('ul').find_all('li')[1].find('a').attrs['href']
            title = li_list[j].find('div', class_='row').find('span').find('a').find('span').text.strip()
            pdf_url = 'https://www.bok.or.kr' + link

            response = requests.get(pdf_url)
            with open(f'pdf_files/{title}.pdf', 'wb') as f:
                f.write(response.content)

            print(f"{title} 다운로드 완료")

        except Exception as e:
            print("Error:", e)

In [13]:
# tsv 파일로 생성

import fitz
import os
import pandas as pd
import re

def extract_date_from_filename(file_name):
    # 파일 이름에서 날짜 정보 추출
    match = re.search(r'(\d{4}\.\d{1,2}\.\d{1,2})', file_name)
    if match:
        return match.group(1)
    else:
        return None

folder_path = "./minutes_pdfs_1"
tsv_file = "minutes_re_1.tsv"

data = []
file_list = os.listdir(folder_path)

for file_name in file_list:
    file_path = os.path.join(folder_path, file_name)
    
    # 파일인 경우에만 처리
    if os.path.isfile(file_path) and file_name.lower().endswith('.pdf'):
        try:
            doc = fitz.open(file_path)
            # 파일 이름에서 날짜 정보 추출
            date_str = extract_date_from_filename(file_name)
            if date_str:
                try:
                    date = pd.to_datetime(date_str, format='%Y.%m.%d')
                except ValueError:
                    print("Invalid date format in file name:", file_name)
                    date = None
            else:
                print("Date information not found in file name:", file_name)
                date = None
            
            # 파일별로 텍스트 추출하여 데이터에 추가 / 2016.6.13. 이전
            text = ""
            for i, page in enumerate(doc):
                if i >= 1:  # 두 번째 페이지부터 시작
                    text += page.get_text()

            # 줄 바꿈 문자를 공백으로 대체
            text = text.replace('\n', ' ')

            # 날짜와 텍스트를 데이터에 추가
            data.append({"날짜": date, "내용": text})
            doc.close()
        except Exception as e:
            print("Error processing file:", file_name)
            print(e)
            continue  # 오류 발생한 파일 건너뛰기

df = pd.DataFrame(data)
df.to_csv(tsv_file, sep='\t', index=False)

print("TSV 파일이 생성되었습니다:", tsv_file)

TSV 파일이 생성되었습니다: minutes_re_1.tsv


In [ ]:
# tsv 파일로 생성

import fitz
import os
import pandas as pd
import re

def extract_date_from_filename(file_name):
    # 파일 이름에서 날짜 정보 추출
    match = re.search(r'(\d{4}\.\d{1,2}\.\d{1,2})', file_name)
    if match:
        return match.group(1)
    else:
        return None

folder_path = "./minutes_pdfs_2"
tsv_file = "minutes_re_2.tsv"

data = []
file_list = os.listdir(folder_path)

for file_name in file_list:
    file_path = os.path.join(folder_path, file_name)
    
    # 파일인 경우에만 처리
    if os.path.isfile(file_path) and file_name.lower().endswith('.pdf'):
        try:
            doc = fitz.open(file_path)
            # 파일 이름에서 날짜 정보 추출
            date_str = extract_date_from_filename(file_name)
            if date_str:
                try:
                    date = pd.to_datetime(date_str, format='%Y.%m.%d')
                except ValueError:
                    print("Invalid date format in file name:", file_name)
                    date = None
            else:
                print("Date information not found in file name:", file_name)
                date = None
            
            # 파일별로 텍스트 추출하여 데이터에 추가 / 2016.6.13. 이후
            text = ""
            for i, page in enumerate(doc):
                if i >= 2:  # 두 번째 페이지부터 시작
                    text += page.get_text()

            # 줄 바꿈 문자를 공백으로 대체
            text = text.replace('\n', ' ')

            # 날짜와 텍스트를 데이터에 추가
            data.append({"날짜": date, "내용": text})
            doc.close()
        except Exception as e:
            print("Error processing file:", file_name)
            print(e)
            continue  # 오류 발생한 파일 건너뛰기

df = pd.DataFrame(data)
df.to_csv(tsv_file, sep='\t', index=False)

print("TSV 파일이 생성되었습니다:", tsv_file)

In [15]:
df

,날짜,내용
0,2013-06-13,- 2 - ( 3 ) 위원 토의내용 국내외 경제동향 과 관련하여 일부 위원은 최근 ...
1,2013-06-27,- 2 - (２) 위원 토의내용 일부 위원은 최근 은행의 중소기업대출이 정부의 중소...
2,2013-07-11,"- 2 - ( 3 ) 위원 토의내용 국내외 경제동향 과 관련하여, 일부 위원은 이번..."
3,2013-07-25,- 2 - (２) 위원 토의내용 일부 위원은 금융기관들이 당행 공개시장조작 대상기관...
4,2013-08-08,"- 2 - ( 3 ) 위원 토의내용 국내외 경제동향 과 관련하여, 일부 위원은 국내..."
...,...,...
154,2023-04-11,- 2 - ( 3 ) 위원 토의내용 전일 개최된 동향보고회의에서는 다음과 같은 토의...
155,2024-01-11,- 2 - ( 3 ) 위원 토의내용 전일 개최된 동향보고회의에서는 다음과 같은 토의...
156,2024-01-25,- 2 - 제안설명에 이어 관련 부서는 12월 20일 및 1월 17일 위원협의회에서...
157,2024-02-08,- 2 - (２) 위원 토의내용 제안설명에 이어 관련부서는 ｢발권규정｣ 개정 내용과...
